In [ ]:
!pip install torch torchvision matplotlib tqdm

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torchvision.models as models

# transformations

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 Dataset

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:04<00:00, 35136492.85it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Dataloaders


In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to modify the final layer of the model

In [16]:
def modify_model(model, num_classes):
        if hasattr(model, 'fc'):
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes)
        elif hasattr(model, 'classifier'):
            if isinstance(model.classifier, nn.Sequential):
                num_ftrs = model.classifier[-1].in_features
                model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
            else:
                num_ftrs = model.classifier.in_features
                model.classifier = nn.Linear(num_ftrs, num_classes)
        return model

# Function to train the model


In [11]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Function to evaluate the model


In [12]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# List of models to train


In [13]:
models_to_train = [
    ("ResNet18", models.resnet18(pretrained=True)),
    ("VGG16", models.vgg16(pretrained=True)),
    ("MobileNetV2", models.mobilenet_v2(pretrained=True))
]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 81.1MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed 

In [14]:
results = []

for model_name, model in models_to_train:
    print(f"\nTraining {model_name}...")
    model = modify_model(model, num_classes=10)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_model(model, train_loader, criterion, optimizer, device, epochs=5)

    accuracy = evaluate_model(model, test_loader, device)
    results.append((model_name, accuracy))


Training ResNet18...


Epoch 1/5: 100%|██████████| 1563/1563 [02:24<00:00, 10.82it/s]


Epoch 1, Loss: 0.6822


Epoch 2/5: 100%|██████████| 1563/1563 [02:23<00:00, 10.89it/s]


Epoch 2, Loss: 0.3985


...

Epoch 3, Loss: 0.3570


Epoch 4/5: 100%|██████████| 1563/1563 [03:03<00:00,  8.53it/s]


Epoch 4, Loss: 0.3128


Epoch 5/5: 100%|██████████| 1563/1563 [03:03<00:00,  8.53it/s]


Epoch 5, Loss: 0.2755
Test Accuracy: 89.28%


# Accuracy Table 

| **Model**           | **Test Accuracy (%)** |
|----------------------|-----------------------|
| ResNet18            | 88.50%                  |
| VGG16               | 63.51%                  |
| MobileNetV2         | 89.28%                  |